In [41]:
import pandas as pd
import os
import random

In [42]:
# Hardcode your file paths here
input_path = '../data/real_world_data/physionet.org/files/mimiciv/3.0/icu/'
output_path = '../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/'  # Path to save the filtered file
file_name = 'icustays.csv.gz'
file_path = os.path.join(input_path, file_name)


In [55]:
# Step 1: Load unique subject IDs
def load_subject_ids(file_path, subject_id_col='subject_id', chunksize=100000):
    """Loads subject IDs from a .csv.gz file in chunks."""
    subject_ids = set()  # Use a set to avoid duplicates
    
    try:
        # Load the file in chunks and collect all subject_ids
        for chunk in pd.read_csv(file_path, compression='gzip', chunksize=chunksize):
            subject_ids.update(chunk[subject_id_col].unique())  # Add unique subject_ids to the set
        
        return list(subject_ids)  # Convert to a list to make it easier to sample
    except Exception as e:
        print(f"Error loading subject IDs from {file_path}: {e}")
        return None

# Step 2: Select a random 5% of subject IDs
def select_random_subject_ids(subject_ids, percentage=5):
    """Selects a random 5% sample of subject IDs."""
    sample_size = int(len(subject_ids) * (percentage / 100))
    return random.sample(subject_ids, sample_size)

def load_single_csv_gz(file_path, patient_ids, patient_id_col='subject_id', chunksize=100000):
    """Loads a single .csv.gz file in chunks and returns a DataFrame, filtering by patient_ids if the column exists."""
    filtered_data = []
    all_data = []  # To collect data regardless of patient_id presence
    patient_id_found = False  # Flag to check if patient_id column exists

    try:
        # Read the file in chunks
        for chunk in pd.read_csv(file_path, compression='gzip', chunksize=chunksize):
            # Display a preview of the first few rows of the chunk
            print(chunk.head())  # Show the first few rows to inspect data
            
            all_data.append(chunk)  # Collect all chunks
            
            # Check for the presence of the patient_id column
            if patient_id_col in chunk.columns:
                patient_id_found = True  # Mark that we've found the patient_id column
                
                # Filter the chunk by patient_ids
                filtered_chunk = chunk[chunk[patient_id_col].isin(patient_ids)]
                filtered_data.append(filtered_chunk)

        # Concatenate all collected chunks into one DataFrame
        if all_data:
            all_data_df = pd.concat(all_data, ignore_index=True)  # Return the full DataFrame regardless

        # If the patient_id column was found and filtered data exists, return that
        if patient_id_found and filtered_data:
            return pd.concat(filtered_data, ignore_index=True), all_data_df
        else:
            return pd.DataFrame(), all_data_df  # Return empty DataFrame for filtered, but return all data

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None, None  # Return None if there's an error

    
# Step 2: Save the filtered DataFrame to a .csv.gz file
def save_filtered_df(filtered_df, output_path):
    """Saves a pandas DataFrame to a .csv.gz file."""
    if filtered_df is not None and not filtered_df.empty:
        
        # Ensure the target folder exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        filtered_df.to_csv(output_path, compression='gzip', index=False)
        print(f"Filtered data saved to {output_path}")
    else:
        print(f"No data to save for {output_path}")
        
        
# Step 3: Load a .csv.gz file and filter by the selected patient IDs
# def load_single_csv_gz(file_path, patient_ids, patient_id_col='subject_id', chunksize=100000):
#    """Loads a single .csv.gz file in chunks and filters by patient_ids."""
#    filtered_data = []
#    
#    try:
#        # Read the file in chunks and filter by patient_ids
#        for chunk in pd.read_csv(file_path, compression='gzip', chunksize=chunksize):
#            filtered_chunk = chunk[chunk[patient_id_col].isin(patient_ids)]
#            filtered_data.append(filtered_chunk)
#        
#        if filtered_data:
#            # Concatenate all filtered chunks into one DataFrame
#            return pd.concat(filtered_data, ignore_index=True)
#        else:
#            return pd.DataFrame()  # Return an empty DataFrame if no matches found
#    except Exception as e:
#        print(f"Error processing {file_path}: {e}")
#        return None

## Step 4: Save the filtered DataFrame to a .csv.gz file
#def save_filtered_df(filtered_df, output_path):
#    """Saves a pandas DataFrame to a .csv.gz file."""
#    if filtered_df is not None and not filtered_df.empty:
#        
#        # Ensure the target folder exists
#       os.makedirs(os.path.dirname(output_path), exist_ok=True)
#        
#        filtered_df.to_csv(output_path, compression='gzip', index=False)
#        print(f"Filtered data saved to {output_path}")
#    else:
#        print(f"No data to save for {output_path}")



In [5]:
# Step 1: Load subject IDs from the file
subject_ids = load_subject_ids(file_path)

if subject_ids:
    # Step 2: Select 5% of the subject IDs randomly
    selected_subject_ids = select_random_subject_ids(subject_ids, percentage=5)
    print(f"Selected {len(selected_subject_ids)} subject IDs from {len(subject_ids)} total.")

    # Step 3: Filter the icustays file by these subject IDs
    filtered_df = load_single_csv_gz(file_path, selected_subject_ids)

    # Step 4: Save the filtered data
    output_file = os.path.join(output_path, file_name)
    save_filtered_df(filtered_df, output_file)
else:
    print(f"No subject IDs found in {file_path}")

Selected 3268 subject IDs from 65366 total.
Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/icustays.csv.gz


In [35]:
file_name = 'inputevents.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [36]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/inputevents.csv.gz


In [23]:
file_name = 'ingredientevents.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [24]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/ingredientevents.csv.gz


In [25]:
file_name = 'outputevents.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [26]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/outputevents.csv.gz


In [27]:
file_name = 'procedureevents.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [28]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/procedureevents.csv.gz


In [29]:
file_name = 'chartevents.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [30]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/chartevents.csv.gz


In [31]:
file_name = 'datetimeevents.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [32]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Filtered data saved to ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/datetimeevents.csv.gz


In [38]:
file_name = 'd_items.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [39]:
# Load and filter the new file by the selected subject IDs
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
save_filtered_df(filtered_df, new_output_path)

Error processing ../data/real_world_data/physionet.org/files/mimiciv/3.0/icu/d_items.csv.gz: 'subject_id'
No data to save for ../data/real_world_data/physionet.org_small/files/mimiciv/3.0/icu/d_items.csv.gz


## Load other data not matchable

In [56]:
file_name = 'd_items.csv.gz'
new_file_path = os.path.join(input_path, file_name)
new_output_path = os.path.join(output_path, file_name)

In [62]:
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)

# Save the filtered data
# save_filtered_df(filtered_df, new_output_path)

   itemid                    label        abbreviation         linksto  \
0  220001             Problem List        Problem List     chartevents   
1  220003       ICU Admission date  ICU Admission date  datetimeevents   
2  220045               Heart Rate                  HR     chartevents   
3  220046  Heart rate Alarm - High     HR Alarm - High     chartevents   
4  220047   Heart Rate Alarm - Low      HR Alarm - Low     chartevents   

              category unitname     param_type  lownormalvalue  \
0              General      NaN           Text             NaN   
1                  ADT      NaN  Date and time             NaN   
2  Routine Vital Signs      bpm        Numeric             NaN   
3               Alarms      bpm        Numeric             NaN   
4               Alarms      bpm        Numeric             NaN   

   highnormalvalue  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  


In [63]:
filtered_df

(Empty DataFrame
 Columns: []
 Index: [],
       itemid                          label                   abbreviation  \
 0     220001                   Problem List                   Problem List   
 1     220003             ICU Admission date             ICU Admission date   
 2     220045                     Heart Rate                             HR   
 3     220046        Heart rate Alarm - High                HR Alarm - High   
 4     220047         Heart Rate Alarm - Low                 HR Alarm - Low   
 ...      ...                            ...                            ...   
 4090  230172               Patient Reversed               Patient Reversed   
 4091  230173  Patient - Fast Track Protocol  Patient - Fast Track Protocol   
 4092  230174              Nerve block in OR              Nerve block in OR   
 4093  230176       IUC Stabilization Device       IUC Stabilization Device   
 4094  230177             CRRT - Filter Type             CRRT - Filter Type   
 
        

## Load Hosp data

In [65]:
# Hardcode your file paths here
input_path_hosp = '../data/real_world_data/physionet.org/files/mimiciv/3.0/hosp/'
output_path_hosp = '../data/real_world_data/physionet.org_small/files/mimiciv/3.0/hosp/'  # Path to save the filtered file
# file_path = os.path.join(input_path, file_name)


In [66]:
file_name = 'patients.csv.gz'
new_file_path = os.path.join(input_path_hosp, file_name)
new_output_path = os.path.join(output_path_hosp, file_name)

In [71]:
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)[0]
save_filtered_df(filtered_df, new_output_path)

   subject_id gender  anchor_age  anchor_year anchor_year_group         dod
0    10000032      F          52         2180       2014 - 2016  2180-09-09
1    10000048      F          23         2126       2008 - 2010         NaN
2    10000058      F          33         2168       2020 - 2022         NaN
3    10000068      F          19         2160       2008 - 2010         NaN
4    10000084      M          72         2160       2017 - 2019  2161-02-13
        subject_id gender  anchor_age  anchor_year anchor_year_group  \
100000    12755701      F          48         2156       2011 - 2013   
100001    12755708      M          52         2174       2017 - 2019   
100002    12755713      F          44         2165       2020 - 2022   
100003    12755718      F          33         2167       2011 - 2013   
100004    12755719      F          64         2164       2011 - 2013   

               dod  
100000         NaN  
100001         NaN  
100002         NaN  
100003         NaN  
100004

In [72]:
file_name = 'admissions.csv.gz'
new_file_path = os.path.join(input_path_hosp, file_name)
new_output_path = os.path.join(output_path_hosp, file_name)

In [73]:
filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)[0]
save_filtered_df(filtered_df, new_output_path)

   subject_id   hadm_id            admittime            dischtime deathtime  \
0    10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00       NaN   
1    10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00       NaN   
2    10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00       NaN   
3    10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00       NaN   
4    10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P49AFC  TRANSFER FROM HOSPITAL   
1        EW EMER.            P784FA          EMERGENCY ROOM   
2        EW EMER.            P19UTS          EMERGENCY ROOM   
3        EW EMER.            P06OTX          EMERGENCY ROOM   
4  EU OBSERVATION            P39NWO          EMERGENCY ROOM   

  discharge_location insurance language marital_status   race  \
0               HOME  Medicaid  English        WIDOWED  WHITE   
1               

        subject_id   hadm_id            admittime            dischtime  \
500000    19145989  27551858  2139-01-19 22:50:00  2139-01-21 12:01:00   
500001    19146037  22167747  2159-12-30 08:00:00  2160-01-08 15:15:00   
500002    19146037  24143387  2159-10-30 13:53:00  2159-11-13 15:25:00   
500003    19146129  22070441  2137-01-10 14:00:00  2137-01-11 00:01:00   
500004    19146129  26199801  2137-02-05 08:00:00  2137-02-05 17:30:00   

       deathtime               admission_type admit_provider_id  \
500000       NaN               EU OBSERVATION            P98ML1   
500001       NaN                       URGENT            P838Z2   
500002       NaN                 DIRECT EMER.            P838Z2   
500003       NaN  SURGICAL SAME DAY ADMISSION            P10JSY   
500004       NaN  SURGICAL SAME DAY ADMISSION            P10JSY   

        admission_location discharge_location insurance language  \
500000      EMERGENCY ROOM                NaN  Medicaid  English   
500001  PHYSICIA

In [74]:
file_name = 'diagnoses_icd.csv.gz'
new_file_path = os.path.join(input_path_hosp, file_name)
new_output_path = os.path.join(output_path_hosp, file_name)


filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)[0]
save_filtered_df(filtered_df, new_output_path)

   subject_id   hadm_id  seq_num icd_code  icd_version
0    10000032  22595853        1     5723            9
1    10000032  22595853        2    78959            9
2    10000032  22595853        3     5715            9
3    10000032  22595853        4    07070            9
4    10000032  22595853        5      496            9
        subject_id   hadm_id  seq_num icd_code  icd_version
100000    10165220  23060728        6     2883            9
100001    10165220  23060728        7     6824            9
100002    10165220  23060728        8     7318            9
100003    10165220  23060728        9    V5867            9
100004    10165220  23060728       10    36201            9
        subject_id   hadm_id  seq_num icd_code  icd_version
200000    10326564  24968587       33     F329           10
200001    10326564  24968587       34     Y839           10
200002    10326564  24968587       35   Y92129           10
200003    10326564  24968587       36     Z931           10
200004    

         subject_id   hadm_id  seq_num icd_code  icd_version
2400000    13766439  21373244        2    66932            9
2400001    13766439  21373244        3    64841            9
2400002    13766439  21373244        4    30000            9
2400003    13766439  21373244        5    64911            9
2400004    13766439  21373244        6    27800            9
         subject_id   hadm_id  seq_num icd_code  icd_version
2500000    13927229  21351110        4     C109           10
2500001    13927229  21351110        5     R112           10
2500002    13927229  21351110        6     M479           10
2500003    13927229  21351110        7     M170           10
2500004    13927229  21351110        8     N400           10
         subject_id   hadm_id  seq_num icd_code  icd_version
2600000    14081759  28736891       16     D649           10
2600001    14081759  28736891       17    G8929           10
2600002    14081759  28736891       18    G4700           10
2600003    14081759  287

         subject_id   hadm_id  seq_num icd_code  icd_version
4700000    17386656  21483965        7   Z87891           10
4700001    17386656  23977109        1     N178           10
4700002    17386656  23977109        2      G92           10
4700003    17386656  23977109        3     E870           10
4700004    17386656  23977109        4     I130           10
         subject_id   hadm_id  seq_num icd_code  icd_version
4800000    17539971  24697837       17    Z9884           10
4800001    17539971  24697837       18   Z20822           10
4800002    17539971  26785055        1    Z5112           10
4800003    17539971  26785055        2    C7802           10
4800004    17539971  26785055        3    C7971           10
         subject_id   hadm_id  seq_num icd_code  icd_version
4900000    17703631  23466885        2    42822            9
4900001    17703631  23466885        3     2761            9
4900002    17703631  23466885        4    59960            9
4900003    17703631  234

In [ ]:
file_name = 'diagnoses_icd.csv.gz'
new_file_path = os.path.join(input_path_hosp, file_name)
new_output_path = os.path.join(output_path_hosp, file_name)


filtered_df = load_single_csv_gz(new_file_path, selected_subject_ids)[0]
save_filtered_df(filtered_df, new_output_path)